In [ ]:
# Scenario Configuration Structure
import numpy as np

def create_scenario(name, **kwargs):
    """Create a scenario configuration with default values that can be overridden"""
    default_config = {
        'name': name,
        'adjustment_window': 4000,
        'num_blocks': 50000,
        'starting_difficulty': 4137588815064,
        'difficulty_growth_factor': 0.0001,
        'start_k_quai': 545e15,
        'k_quai_alpha': 1/1000,
        'target_qi_percentage': 500,
        'chosen_qi_percentage': 0,
        'qi_choice_alpha': 1/500,
        'quai_choices': 2000,
        'qi_choices': 2000,
        'slip': 0.1,
        'slip_alpha': 1/5,
        'qi_percentage_type': 'linear',  # 'linear', 'constant', 'step', 'custom'
        'qi_percentage_custom': None  # for custom qi_percentage_series
    }
    
    # Override defaults with provided kwargs
    for key, value in kwargs.items():
        if key in default_config:
            default_config[key] = value
        else:
            print(f"Warning: Unknown parameter '{key}' in scenario '{name}'")
    
    return default_config

def generate_qi_percentage_series(config):
    """Generate qi_percentage_series based on configuration"""
    num_blocks = config['num_blocks']
    target_qi = config['target_qi_percentage']
    chosen_qi = config['chosen_qi_percentage']
    
    if config['qi_percentage_type'] == 'linear':
        # Current behavior - linear ramp
        series = np.linspace(chosen_qi, target_qi, 15000).tolist()
        if len(series) < num_blocks:
            series.extend([target_qi] * (num_blocks - len(series)))
        return series[:num_blocks]
    
    elif config['qi_percentage_type'] == 'constant':
        # Constant percentage throughout
        return [chosen_qi] * num_blocks
    
    elif config['qi_percentage_type'] == 'step':
        # Step function at midpoint
        midpoint = num_blocks // 2
        return [chosen_qi] * midpoint + [target_qi] * (num_blocks - midpoint)
    
    elif config['qi_percentage_type'] == 'custom' and config['qi_percentage_custom']:
        # Custom series provided
        series = config['qi_percentage_custom'][:num_blocks]
        if len(series) < num_blocks:
            series.extend([series[-1]] * (num_blocks - len(series)))
        return series
    
    else:
        # Default to linear
        return generate_qi_percentage_series({**config, 'qi_percentage_type': 'linear'})

In [ ]:
def run_simulation(config):
    """Run the simulation with given configuration and return results"""
    import numpy as np
    import math
    
    # Extract parameters from config
    adjustment_window = config['adjustment_window']
    num_blocks = config['num_blocks']
    starting_difficulty = config['starting_difficulty']
    difficulty_growth_factor = config['difficulty_growth_factor']
    start_k_quai = config['start_k_quai']
    k_quai_alpha = config['k_quai_alpha']
    target_qi_percentage = config['target_qi_percentage']
    qi_choice_alpha = config['qi_choice_alpha']
    quai_choices = config['quai_choices']
    qi_choices = config['qi_choices']
    slip = config['slip']
    slip_alpha = config['slip_alpha']
    
    # Generate qi_percentage_series based on config
    qi_percentage_series = generate_qi_percentage_series(config)
    
    # Initialize arrays
    block_difficulties = [starting_difficulty]
    d_ema = block_difficulties[0]
    k_quai_series = [start_k_quai]
    d_ema_series = [d_ema]
    d_star_series = [d_ema]
    qi_percentage = 0
    
    # Run simulation
    for n in range(0, num_blocks-1):
        # Update d_ema (ema of average block difficulty)
        d_ema = ((adjustment_window-1) * d_ema + block_difficulties[n]) / adjustment_window
        d_ema_series.append(d_ema)

        d_star = d_ema

        # update d_star based on qi_percentage
        if qi_percentage_series[n] > target_qi_percentage:
            d_star += qi_choice_alpha * (qi_percentage_series[n] - target_qi_percentage) * d_star
        elif qi_percentage_series[n] < target_qi_percentage:
            d_star -= qi_choice_alpha/3 * (target_qi_percentage - qi_percentage_series[n]) * d_star

        # update the d_star based on the realized and actual amount of conversion value
        if quai_choices > qi_choices:
            d_star -= slip_alpha/3 * (slip * d_star)
        elif quai_choices < qi_choices:
            d_star += slip_alpha * (slip * d_star)

        d_star_series.append(d_star)

        d_star_mean = 0
        # Update d* as the simple average of the last 4000 d_ema values (rolling window)
        if n >= adjustment_window:
            d_star_mean = np.mean(d_star_series[n-adjustment_window+1:n+1])
        else:
            d_star_mean = np.mean(d_star_series[:n+1])

        # k_quai update formula
        k_quai_new = k_quai_series[-1] + k_quai_series[-1] * k_quai_alpha * (d_star_mean/d_ema - 1)
        k_quai_series.append(k_quai_new)

        # Update difficulty based on k_quai change
        if k_quai_series[-1] > k_quai_series[-2]:
            new_difficulty = block_difficulties[-1] * (1+difficulty_growth_factor)
            block_difficulties.append(new_difficulty)
        else:
            new_difficulty = block_difficulties[-1] * (1-difficulty_growth_factor)
            block_difficulties.append(new_difficulty)
    
    # Calculate metrics
    percentage_change = (k_quai_series[-1] - k_quai_series[0]) * 100 / k_quai_series[0]
    initial_emission = start_k_quai * math.log2(starting_difficulty)
    final_emission = k_quai_series[-1] * math.log2(block_difficulties[-1])
    emission_reduction = (initial_emission - final_emission) / initial_emission
    
    # Return results
    return {
        'config': config,
        'block_difficulties': block_difficulties,
        'k_quai_series': k_quai_series,
        'd_ema_series': d_ema_series,
        'd_star_series': d_star_series,
        'qi_percentage_series': qi_percentage_series,
        'metrics': {
            'initial_k_quai': k_quai_series[0],
            'final_k_quai': k_quai_series[-1],
            'percentage_change_k_quai': percentage_change,
            'initial_emission': initial_emission,
            'final_emission': final_emission,
            'emission_reduction_percentage': emission_reduction,
            'final_d_star': d_star_series[-1],
            'final_d_ema': d_ema_series[-1]
        }
    }

In [ ]:
def plot_scenario_results(results, title_suffix=""):
    """Plot the results of a single scenario simulation"""
    import matplotlib.pyplot as plt
    
    config = results['config']
    num_blocks = config['num_blocks']
    
    fig, axs = plt.subplots(3, 1, figsize=(14, 14), sharex=True)
    
    block_numbers = list(range(num_blocks))
    
    # Plot 1: Block Difficulty and k_quai
    color1 = 'tab:blue'
    color2 = 'tab:green'
    ax1 = axs[0]
    lns1 = ax1.plot(block_numbers, results['block_difficulties'], color=color1, label='Block Difficulty')
    ax1.set_ylabel('Block Difficulty', color=color1)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.legend(loc='upper left')
    ax1b = ax1.twinx()
    lns2 = ax1b.plot(block_numbers, results['k_quai_series'], color=color2, label='k_quai')
    ax1b.set_ylabel('k_quai', color=color2)
    ax1b.tick_params(axis='y', labelcolor=color2)
    lns = lns1 + lns2
    labels = [l.get_label() for l in lns]
    ax1.legend(lns, labels, loc='upper left')
    ax1.set_title(f'Block Difficulty and k_quai Movement - {config["name"]}{title_suffix}')
    
    # Plot 2: d* and d_ema
    block_numbers_d = list(range(len(results['d_star_series'])))
    ax2 = axs[1]
    ax2.plot(block_numbers_d, results['d_star_series'], label='d* (Target Difficulty)', color='tab:orange')
    ax2.plot(block_numbers_d, results['d_ema_series'], label='d_ema (EMA Difficulty)', color='tab:purple')
    ax2.set_ylabel('Difficulty')
    ax2.set_title(f'd* (Target Difficulty) and d_ema (EMA Difficulty) - {config["name"]}{title_suffix}')
    ax2.legend()
    
    # Plot 3: qi_percentage_series
    block_numbers_qi = list(range(len(results['qi_percentage_series'])))
    ax3 = axs[2]
    ax3.plot(block_numbers_qi, results['qi_percentage_series'], label='qi_percentage', color='tab:red')
    ax3.axhline(y=config['target_qi_percentage'], color='tab:gray', linestyle='--', label='Target qi_percentage')
    ax3.set_xlabel('Block Number')
    ax3.set_ylabel('qi_percentage')
    ax3.set_title(f'qi_percentage Over Block Number - {config["name"]}{title_suffix}')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics
    metrics = results['metrics']
    print(f"\n=== {config['name']} Results ===")
    print(f"Initial k_quai: {metrics['initial_k_quai']:.2e}")
    print(f"Final k_quai: {metrics['final_k_quai']:.2e}")
    print(f"Percentage change in k_quai: {metrics['percentage_change_k_quai']:.2f}%")
    print(f"Initial emission per block: {metrics['initial_emission']:.2e}")
    print(f"Final emission per block: {metrics['final_emission']:.2e}")
    print(f"Emission reduction: {metrics['emission_reduction_percentage']:.2f}%")
    print(f"Final d* (target difficulty): {metrics['final_d_star']:.2e}")
    print(f"Final d_ema: {metrics['final_d_ema']:.2e}")

def compare_scenarios(results_list, metrics_to_compare=None):
    """Compare multiple scenario results in a summary table"""
    import pandas as pd
    
    if metrics_to_compare is None:
        metrics_to_compare = ['final_k_quai', 'percentage_change_k_quai', 'emission_reduction_percentage']
    
    comparison_data = []
    for results in results_list:
        row = {'Scenario': results['config']['name']}
        for metric in metrics_to_compare:
            if metric in results['metrics']:
                row[metric] = results['metrics'][metric]
        comparison_data.append(row)
    
    df = pd.DataFrame(comparison_data)
    return df

In [ ]:
# Define multiple scenarios to test
scenarios = [
    # Scenario A: Current behavior - linear ramp to target qi percentage
    create_scenario("Linear Ramp to Target", 
                   qi_percentage_type='linear'),
    
    # Scenario B: No qi adoption - everyone stays with quai
    create_scenario("No Qi Adoption", 
                   chosen_qi_percentage=0,
                   target_qi_percentage=0,
                   qi_percentage_type='constant'),
    
    # Scenario C: Immediate full qi adoption
    create_scenario("Immediate Full Qi", 
                   chosen_qi_percentage=500,
                   target_qi_percentage=500,
                   qi_percentage_type='constant'),
    
    # Scenario D: Step function - sudden change at midpoint
    create_scenario("Sudden Qi Adoption", 
                   qi_percentage_type='step'),
    
    # Scenario E: High k_quai alpha (more responsive controller)
    create_scenario("High Responsiveness", 
                   k_quai_alpha=1/100,  # 10x more responsive
                   qi_percentage_type='linear'),
    
    # Scenario F: Low k_quai alpha (less responsive controller)
    create_scenario("Low Responsiveness", 
                   k_quai_alpha=1/10000,  # 10x less responsive
                   qi_percentage_type='linear'),
    
    # Scenario G: High slip (expensive conversion)
    create_scenario("High Conversion Cost", 
                   slip=0.5,  # 50% slip instead of 10%
                   qi_percentage_type='linear'),
    
    # Scenario H: Qi dominant choice (more qi choices than quai)
    create_scenario("Qi Dominant", 
                   quai_choices=1000,
                   qi_choices=3000,
                   qi_percentage_type='linear'),
    
    # Scenario I: Very high target qi percentage
    create_scenario("High Qi Target", 
                   target_qi_percentage=1000,  # 10% instead of 5%
                   qi_percentage_type='linear'),
    
    # Scenario J: Short simulation for quick testing
    create_scenario("Quick Test", 
                   num_blocks=10000,
                   qi_percentage_type='linear')
]

print(f"Defined {len(scenarios)} scenarios:")
for i, scenario in enumerate(scenarios, 1):
    print(f"{i}. {scenario['name']}")

In [ ]:
def run_all_scenarios(scenarios, plot_each=True, show_comparison=True):
    """Run all scenarios and optionally plot results and show comparison"""
    results_list = []
    
    print("Running scenarios...")
    for i, scenario in enumerate(scenarios, 1):
        print(f"Running scenario {i}/{len(scenarios)}: {scenario['name']}...")
        results = run_simulation(scenario)
        results_list.append(results)
        
        if plot_each:
            plot_scenario_results(results)
    
    if show_comparison:
        print("\n" + "="*50)
        print("SCENARIO COMPARISON")
        print("="*50)
        comparison_df = compare_scenarios(results_list)
        print(comparison_df.to_string(index=False))
    
    return results_list

def run_specific_scenarios(scenario_indices, scenarios=None):
    """Run specific scenarios by index"""
    if scenarios is None:
        scenarios = globals()['scenarios']  # Use the global scenarios list
    
    selected_scenarios = [scenarios[i] for i in scenario_indices]
    return run_all_scenarios(selected_scenarios)

# Example usage functions:
def run_quick_test():
    """Run just the quick test scenario"""
    return run_specific_scenarios([9])  # Quick Test is index 9

def run_responsiveness_comparison():
    """Compare different k_quai_alpha values"""
    return run_specific_scenarios([0, 4, 5])  # Linear, High, Low responsiveness

def run_adoption_patterns():
    """Compare different qi adoption patterns"""
    return run_specific_scenarios([0, 1, 2, 3])  # Linear, No adoption, Immediate, Step

# Usage Examples

Now you can easily run different scenarios! Here are some examples:

```python
# Run a quick test (just 10k blocks)
results = run_quick_test()

# Compare responsiveness settings
results = run_responsiveness_comparison()

# Compare adoption patterns
results = run_adoption_patterns()

# Run all scenarios (warning: takes time and shows lots of plots!)
# results = run_all_scenarios(scenarios, plot_each=True, show_comparison=True)

# Run all scenarios without individual plots (just get comparison table)
# results = run_all_scenarios(scenarios, plot_each=False, show_comparison=True)

# Run specific scenarios by index
# results = run_specific_scenarios([0, 2, 4])  # Linear, Immediate Full Qi, High Responsiveness

# Run a single scenario
# single_result = run_simulation(scenarios[0])
# plot_scenario_results(single_result)
```

## Scenario List:
0. Linear Ramp to Target (your original scenario)
1. No Qi Adoption
2. Immediate Full Qi
3. Sudden Qi Adoption (step function)
4. High Responsiveness (k_quai_alpha=1/100)
5. Low Responsiveness (k_quai_alpha=1/10000)
6. High Conversion Cost (slip=50%)
7. Qi Dominant (more qi choices)
8. High Qi Target (10% target)
9. Quick Test (10k blocks)